heurística // numpy

In [ ]:
import numpy as np
import math

# Supondo que n > 0, V > 0 e v[i] >=  e que todos itens cabem nos recipientes

# Ordenar os itens pelo peso, colocar o primeiro item no primeiro bin, depois ver em qual dos bins já ocupados o próximo item cabe melhor (sobra menos espaço), se não couber em nenhum, coloca no primeiro bin vazio

# Entradas
teste = np.loadtxt("arquivo.txt")
V = int(teste[1])  # Capacidade dos recipientes
B = 0  # Quantidade de recipientes não vazios
v = teste[2:teste.size].astype(int)  # Peso de cada item
n = v.size  # Quantidade de itens
# Ordenando v de forma decrescente (tentar também com outros métodos)
v = -np.sort(-v, kind='heapsort')
# Criando o vetor com os recipientes ainda vazios
S = (np.zeros(n) + V).astype(int)
S[0] -= v[0]  # Colocando o primeiro item no primeiro recipiente
B += 1  # Aumentando a quantidade de recipientes ocupados
for i in v[1:n]:  # Considerar todos os itens que restam (do segundo ao ultimo)
    # R[1] vai indicar em qual recipiente o elemento i cabe deixando menos espaco sobrando
    R = np.array([math.inf, -1])
    for j in range(B):  # Para cada recipiente ja ocupado
        if R[0] > (S[j] - i) and (S[j] - i) >= 0:
            R[0] = S[j] - i
            R[1] = j
    # deu errado (não cabe em nenhum já ocupado) (colocar no próximo)
    if R[0] == math.inf:
        S[B] = S[B] - i
        B = B + 1
    else:  # deu certo (colocar i onde couber melhor)
        S[int(R[1])] = S[int(R[1])] - i
print(B)


modelo // mip

In [ ]:
from mip import Model, xsum, BINARY, INTEGER

L = 10  # Capacidade dos Bins
w = [3, 2, 1, 2, 4, 5, 3, 9, 8, 7, 5, 6, 3, 2, 1, 0, 5, 6, 7, 8, 3, 1, 1, 2, 2, 8,
     8, 9, 5, 9, 5, 8, 7, 6, 3, 4, 9, 8, 7, 6, 5, 5, 5, 10, 3, 1, 3, 1, 3]  # Itens
n = len(w)  # Quantidade de Itens

# creating the model
model = Model()
x = {(i, j): model.add_var(obj=0, var_type=INTEGER, name="x[%d,%d]" % (i, j))
     for i in range(n) for j in range(n)}
y = {j: model.add_var(obj=1, var_type=BINARY, name="y[%d]" % j)
     for j in range(n)}

# constraints
for i in range(n):
    model.add_constr(xsum(x[i, j] for j in range(n)) == 1)
for j in range(n):
    model.add_constr(xsum(w[i] * x[i, j] for i in range(n)) <= L * y[j])

# additional constraints to reduce symmetry
for j in range(1, n):
    model.add_constr(y[j - 1] >= y[j])

# optimizing the model
model.optimize()

# printing the solution
print('')
print('Objective value: {model.objective_value:.3}'.format(**locals()))
print('Solution: ', end='')
for v in model.vars:
    if v.x > 1e-5:
        print('{v.name} = {v.x}'.format(**locals()))
        print('          ', end='')